In [14]:
# Importing necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Step 1: Identify the Objective, Criteria, and Alternatives

### Step 2: Create a Pair-wise Comparison Matrix of the Criteria Selected

In [15]:
# Define criteria and their pairwise comparisons
criteria = ["Serviceability", "Supply Capacity", "Quality", "Cost"]

pairwise_comparison_matrix = [
    [1, 3, 2, 2],
    [1/3, 1, 1/4, 1/4],
    [1/2, 4, 1, 1/2],
    [1/2, 4, 2, 1]
]
ahp_df = pd.DataFrame(data = pairwise_comparison_matrix, index = criteria, columns=criteria)
ahp_df

,Serviceability,Supply Capacity,Quality,Cost
Serviceability,1.000000,3,2.00,2.00
Supply Capacity,0.333333,1,0.25,0.25
Quality,0.500000,4,1.00,0.50
Cost,0.500000,4,2.00,1.00


### Step 3: Find the Priority Index for Each Attribute or Criterion

In [16]:
# We will introduce a function to find the priority index. 
# Then we provide the attributes data to this function.
def ahp_attributes(ahp_df):
    # Creating an array of sum of values in each column
    sum_array = np.array(ahp_df.sum(numeric_only=True))
    # Creating a normalized pairwise comparison matrix.
    # By dividing each column cell value with the sum of the respective column.
    cell_by_sum = ahp_df.div(sum_array,axis=1)
    # Creating Priority index by taking avg of each row
    priority_df = pd.DataFrame(cell_by_sum.mean(axis=1),
                               index=ahp_df.index,columns=['priority index'])
    priority_df = priority_df.transpose()
    return priority_df

In [17]:
# Calling the ahp_attributes function, 
# To return a table with the priority index for each attribute.
priority_index_attr = ahp_attributes(ahp_df)
priority_index_attr

,Serviceability,Supply Capacity,Quality,Cost
priority index,0.398214,0.085119,0.217857,0.29881


The output shows us that serviceability has the highest priority, followed by cost, and supply capacity has the least priority.

In [18]:
def consistency_ratio(priority_index,ahp_df):
    random_matrix = {1:0,2:0,3:0.58,4:0.9,5:1.12,6:1.24,7:1.32,
                     8:1.14,9:1.45,10:1.49,11:1.51,12:1.48,13:1.56,
                     14:1.57,15:1.59,16:1.605,17:1.61,18:1.615,19:1.62,20:1.625}
    # Check for consistency
    consistency_df = ahp_df.multiply(np.array(priority_index.loc['priority index']),axis=1)
    consistency_df['sum_of_col'] = consistency_df.sum(axis=1)
    # To find lambda max
    lambda_max_df = consistency_df['sum_of_col'].div(np.array(priority_index.transpose()
                                                              ['priority index']),axis=0)
    lambda_max = lambda_max_df.mean()
    # To find the consistency index
    consistency_index = round((lambda_max-len(ahp_df.index))/(len(ahp_df.index)-1),3)
    print(f'The Consistency Index is: {consistency_index}')
    # To find the consistency ratio
    consistency_ratio = round(consistency_index/random_matrix[len(ahp_df.index)],3)
    print(f'The Consistency Ratio is: {consistency_ratio}')
    if consistency_ratio<0.1:
        print('The model is consistent')
    else:
        print('The model is not consistent')
        

In [6]:
consistency_ratio(priority_index_attr,ahp_df)

The Consistency Index is: 0.062
The Consistency Ratio is: 0.069
The model is consistent


### Step 5: Find the Priority Index of the Suppliers Based on Each Criterion

In [11]:
import pandas as pd

# Create a dictionary to hold the data
data = {
    ('Serviceability', 'supplier1'): [0.389704, 0.956495, 0.296954],
    ('Serviceability', 'supplier2'): [0.456191, 0.619090, 0.419591],
    ('Serviceability', 'supplier3'): [0.724464, 0.662622, 0.640829],
    ('Supply Capacity', 'supplier1'): [0.389704, 0.956495, 0.296954],
    ('Supply Capacity', 'supplier2'): [0.456191, 0.619090, 0.419591],
    ('Supply Capacity', 'supplier3'): [0.724464, 0.662622, 0.640829],
    ('Quality', 'supplier1'): [0.389704, 0.956495, 0.296954],
    ('Quality', 'supplier2'): [0.456191, 0.619090, 0.419591],
    ('Quality', 'supplier3'): [0.724464, 0.662622, 0.640829],
    ('Cost', 'supplier1'): [0.389704, 0.956495, 0.296954],
    ('Cost', 'supplier2'): [0.456191, 0.619090, 0.419591],
    ('Cost', 'supplier3'): [0.724464, 0.662622, 0.640829]
}

# Create a multi-index DataFrame
df = pd.DataFrame(data)
ahp_df_1 = df.T
ahp_df_1
# # Rename the columns and index
# df.columns.names = ['attributes', 'suppliers']
ahp_df_1.index.names = ['attributes', 'suppliers']

# # Display the DataFrame

ahp_df_1.columns = ['supplier1', 'supplier2', 'supplier3']
ahp_df_1

supplier1  supplier2  supplier3
attributes      suppliers                                 
Serviceability  supplier1   0.389704   0.956495   0.296954
                supplier2   0.456191   0.619090   0.419591
                supplier3   0.724464   0.662622   0.640829
Supply Capacity supplier1   0.389704   0.956495   0.296954
                supplier2   0.456191   0.619090   0.419591
                supplier3   0.724464   0.662622   0.640829
Quality         supplier1   0.389704   0.956495   0.296954
                supplier2   0.456191   0.619090   0.419591
                supplier3   0.724464   0.662622   0.640829
Cost            supplier1   0.389704   0.956495   0.296954
                supplier2   0.456191   0.619090   0.419591
                supplier3   0.724464   0.662622   0.640829

In [12]:
def supplier_priority_index(suppl_attr_df,num_attr,attr_name):
    data_dict = {}
    # To find supplier priority indices
    # Supplier priority for attr 1
    data_dict[f"ahp_df_suppl_{attr_name}"] = suppl_attr_df.loc[attr_name]
    # Creating an array of sum of values in each column
    data_dict[f"sum_array_suppl_{attr_name}"] = np.array(data_dict[
        f"ahp_df_suppl_{attr_name}"].sum(numeric_only=True))
    # Normalised pairwise comparison matrix
    # Dividing each column cell value with the sum of the respective column.
    data_dict[f"norm_mat_suppl_{attr_name}"] = data_dict[
        f"ahp_df_suppl_{attr_name}"].div(data_dict[f"sum_array_suppl_{attr_name}"],axis=1)
    priority_df = pd.DataFrame(data_dict[
        f"norm_mat_suppl_{attr_name}"].mean(axis=1),
                               index=suppl_attr_df.loc[attr_name].index,columns=[attr_name])
    return priority_df

In [13]:
suppl_service_df = supplier_priority_index(ahp_df_1,4,'Serviceability')
suppl_capacity_df = supplier_priority_index(ahp_df_1,4,'Supply Capacity')
suppl_quality_df = supplier_priority_index(ahp_df_1,4,'Quality')
suppl_cost_df = supplier_priority_index(ahp_df_1,4,'Cost')

suppl_df = pd.concat([suppl_service_df,suppl_capacity_df,suppl_quality_df,suppl_cost_df],axis=1)
suppl_norm_df = suppl_df.multiply(np.array(priority_index_attr.loc['priority index']),axis=1)
suppl_norm_df['Sum'] = suppl_norm_df.sum(axis=1)
suppl_norm_df

,Serviceability,Supply Capacity,Quality,Cost,Sum
suppliers,,,,,
supplier1,0.118705,0.025373,0.064942,0.089073,0.298094
supplier2,0.116308,0.024861,0.063630,0.087274,0.292074
supplier3,0.163201,0.034885,0.089285,0.122462,0.409832


As we can see from the above matrix, the sum of weights or ratings is the maximum for Supplier 3